In [9]:
from pprint import pprint
from functools import reduce
import brotli
import pickle


with open("./dump/dbeaver_dbeaver/commits.szhat".replace("\\","/"), "rb") as handler:
    # print(handler.read())
    data = handler.read()
    decompressed_data = brotli.decompress(data)
    obj = pickle.loads(decompressed_data)

# print(c[1].files)
# avg = reduce(lambda x, y: x+y, map(lambda c: c.files, commits))/ len(commits)
# c = commits[0]

# print(c.stats.additions)

In [10]:
print([o for o in obj if o.files is None])

[Commit(sha="fffdc7e483462617869d63f38d4f6473b084c876")]


## Gathering the data

In [2]:
import glob
import pickle
from utils.metrics.contributors import Contributors
from collections import defaultdict
from pprint import pprint
import pandas as pd

repo_dicts = defaultdict(lambda: {})
metric_dicts = defaultdict(lambda: defaultdict(lambda: {}))
for p in glob.glob(".\intermediate\*\*.pickle"):
    with open(p, "rb") as file:
        metric = pickle.load(file)
        dic = metric.__dict__
        repo_name = dic.pop("repo_name")
        metric_type = p.split('\\')[-1].split(".")[0]

        metric_dicts[metric_type][repo_name] |= dic
        repo_dicts[repo_name] |= dic
downloads = [repo for repo in repo_dicts.values() if repo.get("[Releases] Total downloads")]
no_downloads = [repo for repo in repo_dicts.values() if not repo.get("[Releases] Total downloads")]
releases = [repo for repo in repo_dicts.values() if repo.get("[Releases] Count")]
no_releases = [repo for repo in repo_dicts.values() if not repo.get("[Releases] Count")]


data = pd.DataFrame.from_dict(repo_dicts)
downloads_data =  pd.DataFrame.from_dict(downloads)
no_downloads_data =  pd.DataFrame.from_dict(no_downloads)

print(f"Metrics count: {len(data)}")
print(f"Repo    count: {len(data.keys())}")
print(f"Has downloads: {len(downloads_data)}")
print(f"No  downloads: {len(no_downloads_data)}")
print(f"Has Releases: {len(releases)}")
print(f"No  Releases: {len(no_releases)}")
display(data)
data.to_csv("metrics.csv",index=True)

Metrics count: 112
Repo    count: 179
Has downloads: 63
No  downloads: 116
Has Releases: 127
No  Releases: 52


,Activiti/Activiti,airbnb/lottie-android,alibaba/arthas,alibaba/fastjson,alibaba/Sentinel,alibaba/spring-cloud-alibaba,android-async-http/android-async-http,ansible/ansible,apache/beam,apache/cassandra,...,Tencent/tinker,testcontainers/testcontainers-java,TheCherno/Hazel,tony19/logback-android,typelevel/cats,varietywalls/variety,vifm/vifm,Wox-launcher/Wox,zabbix/zabbix,zxing/zxing
[Commits] Count,10768.0,1358.0,1659.000000,3933.0,713.0,1838.0,899.0,NaN,33534.0,27038.0,...,735.0,2947.0,243.0,4238.0,NaN,NaN,11773.0,NaN,NaN,3620.0
[Commits] Days since first,4165.0,1917.0,1169.000000,3757.0,1208.0,1441.0,3918.0,NaN,2525.0,4638.0,...,1878.0,2406.0,1123.0,5660.0,NaN,NaN,4681.0,NaN,NaN,5134.0
[Commits] Days since last,10.0,6.0,8.000000,31.0,58.0,11.0,298.0,NaN,0.0,0.0,...,12.0,10.0,59.0,541.0,NaN,NaN,3.0,NaN,NaN,21.0
[Commits] Total lines added,8310435.0,122432.0,244391.000000,799527.0,167770.0,553069.0,44931.0,NaN,14154262.0,3239411.0,...,110864.0,102479.0,50664.0,493487.0,NaN,NaN,1071378.0,NaN,NaN,831281.0
[Commits] Total lines deleted,5064341.0,59739.0,40864.000000,191777.0,25075.0,303082.0,19781.0,NaN,7319061.0,1712123.0,...,32416.0,38560.0,13563.0,386642.0,NaN,NaN,505776.0,NaN,NaN,617442.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
[Contributors Top-100] Average commits,NaN,NaN,15.505051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[Contributors Top-100] Average participation weeks,NaN,NaN,3.212121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[Contributors Top-100] Average additions,NaN,NaN,2393.969697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[Contributors Top-100] Average deletions,NaN,NaN,653.616162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


[{'[Commits] Count': 10768, '[Commits] Days since first': 4165, '[Commits] Days since last': 10, '[Commits] Total lines added': 8310435, '[Commits] Total lines deleted': 5064341, '[Commits] Average additions': 771.7714524517088, '[Commits] Average deletions': 470.31398588410104, '[Commits] Average files changed': 11.078101783060921, '[Commits] Average message length (chars)': 84.61599182763744, '[Commits] Per day': 5.3096646942800785, '[Commits] Per day (True)': 3.217209441290708, '[Commits] Maximum per day': 79, '[Contributors] Count': 195, '[Contributors Top-10] Average commits': 391.9, '[Contributors Top-10] Average participation weeks': 67.3, '[Contributors Top-10] Average additions': 259476.9, '[Contributors Top-10] Average deletions': 210635.0, '[Contributors Top-10] Average followers': 51.5, '[Contributors Top-10] Average following': 1.3, '[Contributors Top-10] Average public repositories': 41.7, '[Contributors Top-10] Average teams': 0.0, '[Contributors Top-10] Average stars': 

## Calculation correlation

In [31]:
from itertools import combinations_with_replacement

# for data, name in zip([data, downloads_data, no_downloads_data], ["correlations", "downloads_correlations", "no_downloads_correlations"]):
values = {name:data.loc[name] for name in data.index}
correlations = pd.DataFrame(values).corr(method='pearson').values
frame = pd.DataFrame(pd.np.triu(correlations), index=data.index, columns=data.index)
frame.to_csv(f"{name}.csv", index=True)
display(frame)

for g1, g2 in combinations_with_replacement(metric_dicts.items(), 2):
    _, inner_dict1 = g1
    _, inner_dict2 = g2

    res = {name:inner_dict1[name] | inner_dict2[name] for name in inner_dict1}
    data = pd.DataFrame(res).transpose().corr(method='pearson')
    # data = data[data[data.columns] > 0.7]
    metric1_names = list(list(inner_dict1.values())[0].keys())
    metric2_names = list(list(inner_dict2.values())[0].keys())

    df = data[metric1_names].loc[metric2_names]
    display(df)

C:\Users\79134\AppData\Local\Temp/ipykernel_39592/491823013.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  frame = pd.DataFrame(pd.np.triu(correlations), index=data.index, columns=data.index)


,[Commits] Count,[Commits] Days since first,[Commits] Days since last,[Commits] Total lines added,[Commits] Total lines deleted,[Commits] Average additions,[Commits] Average deletions,[Commits] Average files changed,[Commits] Average message length (chars),[Commits] Per day,...,[Workflow Runs] Average failure duration (ms),[Workflow Runs] Average successes per day,[Workflow Runs] Average successes per day (True),[Workflow Runs] Average fails per day,[Workflow Runs] Average fails per day (True),[Contributors Top-100] Average commits,[Contributors Top-100] Average participation weeks,[Contributors Top-100] Average additions,[Contributors Top-100] Average deletions,[Releases] Drafts to total
[Commits] Count,1.0,0.336499,-0.195355,0.266138,0.249127,0.016125,0.066167,0.005149,0.195045,0.785946,...,0.225525,0.406579,0.494770,0.432031,0.610228,0.122066,0.581438,-0.146934,-0.043731,NaN
[Commits] Days since first,0.0,1.000000,-0.043490,0.042705,0.026823,0.027577,0.027220,0.045579,0.136323,-0.019627,...,-0.010945,-0.011939,-0.040345,-0.005489,0.046016,-0.041567,0.150709,-0.260978,-0.135851,NaN
[Commits] Days since last,0.0,0.000000,1.000000,-0.090570,-0.073125,-0.057996,-0.062784,-0.057876,-0.215170,-0.043943,...,-0.116852,-0.143340,-0.129193,-0.171921,-0.134549,0.707419,0.182321,0.957375,0.913535,NaN
[Commits] Total lines added,0.0,0.000000,0.000000,1.000000,0.957869,0.728837,0.876814,0.599793,-0.042469,0.333199,...,0.146762,0.269927,0.190504,0.134702,0.062199,0.730954,0.325282,0.996669,0.988827,NaN
[Commits] Total lines deleted,0.0,0.000000,0.000000,0.000000,1.000000,0.519602,0.730782,0.365614,-0.050851,0.346092,...,0.174730,0.139880,0.097637,0.154582,0.030296,0.710781,0.345103,0.981698,0.998195,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
[Contributors Top-100] Average commits,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.705005,0.745047,0.734092,NaN
[Contributors Top-100] Average participation weeks,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.313115,0.386554,NaN
[Contributors Top-100] Average additions,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.981449,NaN
[Contributors Top-100] Average deletions,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,NaN


,[Commits] Count,[Commits] Days since first,[Commits] Days since last,[Commits] Total lines added,[Commits] Total lines deleted,[Commits] Average additions,[Commits] Average deletions,[Commits] Average files changed,[Commits] Average message length (chars),[Commits] Per day,[Commits] Per day (True),[Commits] Maximum per day
[Commits] Count,1.000000,0.336499,-0.195355,0.266138,0.249127,0.016125,0.066167,0.005149,0.195045,0.785946,0.517350,0.171500
[Commits] Days since first,0.336499,1.000000,-0.043490,0.042705,0.026823,0.027577,0.027220,0.045579,0.136323,-0.019627,0.254840,-0.103137
[Commits] Days since last,-0.195355,-0.043490,1.000000,-0.090570,-0.073125,-0.057996,-0.062784,-0.057876,-0.215170,-0.043943,-0.078263,-0.050727
[Commits] Total lines added,0.266138,0.042705,-0.090570,1.000000,0.957869,0.728837,0.876814,0.599793,-0.042469,0.333199,0.117274,0.409256
[Commits] Total lines deleted,0.249127,0.026823,-0.073125,0.957869,1.000000,0.519602,0.730782,0.365614,-0.050851,0.346092,0.103482,0.471299
[Commits] Average additions,0.016125,0.027577,-0.057996,0.728837,0.519602,1.000000,0.955186,0.972059,-0.058708,0.013640,-0.004196,0.114700
[Commits] Average deletions,0.066167,0.027220,-0.062784,0.876814,0.730782,0.955186,1.000000,0.887336,-0.082552,0.089795,0.009053,0.239984
[Commits] Average files changed,0.005149,0.045579,-0.057876,0.599793,0.365614,0.972059,0.887336,1.000000,-0.061458,-0.030873,0.002065,0.030715
[Commits] Average message length (chars),0.195045,0.136323,-0.215170,-0.042469,-0.050851,-0.058708,-0.082552,-0.061458,1.000000,0.077994,0.093382,0.122175
[Commits] Per day,0.785946,-0.019627,-0.043943,0.333199,0.346092,0.013640,0.089795,-0.030873,0.077994,1.000000,0.257888,0.378347


,[Commits] Count,[Commits] Days since first,[Commits] Days since last,[Commits] Total lines added,[Commits] Total lines deleted,[Commits] Average additions,[Commits] Average deletions,[Commits] Average files changed,[Commits] Average message length (chars),[Commits] Per day,[Commits] Per day (True),[Commits] Maximum per day
[Contributors] Count,0.565188,0.181860,-0.335951,0.108079,0.079905,0.024052,0.031450,0.043140,0.315895,0.388519,0.205384,0.054682
[Contributors Top-10] Average commits,0.726884,0.398101,-0.234490,0.280816,0.294886,0.009453,0.082663,-0.021699,0.208108,0.490986,0.476152,0.213963
[Contributors Top-10] Average participation weeks,0.747812,0.378787,-0.267021,0.187092,0.177209,-0.023945,0.015809,-0.034564,0.350119,0.455388,0.636048,0.118869
[Contributors Top-10] Average additions,0.239692,0.009506,-0.083797,0.843560,0.943726,0.285927,0.523411,0.116445,-0.011486,0.350941,0.113794,0.499237
[Contributors Top-10] Average deletions,0.234849,0.014058,-0.079356,0.842418,0.946094,0.283052,0.524408,0.116652,-0.026185,0.344422,0.107813,0.507023
[Contributors Top-10] Average followers,-0.158032,-0.056297,0.034619,-0.066676,-0.050102,-0.055616,-0.056769,-0.056048,-0.056966,-0.139900,-0.061143,-0.022084
[Contributors Top-10] Average following,-0.219267,-0.216990,0.077168,-0.112634,-0.092593,-0.074712,-0.075504,-0.068111,-0.160613,-0.155100,-0.104845,-0.065972
[Contributors Top-10] Average public repositories,0.011982,0.103722,-0.043101,-0.146043,-0.105537,-0.142816,-0.131303,-0.149137,-0.057472,-0.025206,-0.119629,-0.008987
[Contributors Top-10] Average teams,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
[Contributors Top-10] Average stars,-0.176766,-0.085177,0.061827,-0.084904,-0.069093,-0.064540,-0.070593,-0.062487,-0.083302,-0.144808,-0.068029,-0.021268


,[Commits] Count,[Commits] Days since first,[Commits] Days since last,[Commits] Total lines added,[Commits] Total lines deleted,[Commits] Average additions,[Commits] Average deletions,[Commits] Average files changed,[Commits] Average message length (chars),[Commits] Per day,[Commits] Per day (True),[Commits] Maximum per day
[Forks] Count,0.227290,0.002909,-0.127395,-0.004897,-0.006961,-0.013548,-0.013769,-0.001513,0.068849,0.183390,-0.007731,-0.028942
[Forks] Per day (True),0.172576,-0.124823,-0.139855,-0.015942,-0.016433,-0.018404,-0.021694,-0.005665,0.019636,0.156953,0.011693,-0.028278
[Forks] Max per day,0.044684,0.120748,-0.050137,-0.042220,-0.036293,-0.046288,-0.048407,-0.031344,-0.030184,0.010944,-0.018009,-0.036534


...

,[Releases] Count,[Releases] Tags,[Releases] Total downloads,[Releases] Average body len (chars),[Releases] Average title len (chars),[Releases] Average assets,[Releases] Average asset downloads,[Releases] Average asset size,[Releases] Prereleases to total,[Releases] Per day created (True),[Releases] Per day published (True),[Releases] Downloads per day (True)
[Stars] Count,0.101821,-0.028511,0.248977,0.414560,-0.058565,0.151893,0.160723,-0.022867,-0.041279,0.266208,0.319132,0.331487
[Stars] Per day,-0.062300,0.127002,-0.077081,0.094838,0.400666,-0.088226,0.014557,-0.039103,0.103621,-0.166982,-0.186511,-0.086817
[Stars] Per day (True),0.187378,-0.055193,0.269872,0.219752,-0.078419,0.125487,0.262341,-0.001052,-0.102957,0.307816,0.368185,0.480720
[Stars] Maximum per day,-0.006216,0.208810,-0.108858,0.036122,0.108088,-0.066742,-0.063494,-0.016210,0.083636,-0.119570,-0.142146,-0.114135


,[Releases] Count,[Releases] Tags,[Releases] Total downloads,[Releases] Average body len (chars),[Releases] Average title len (chars),[Releases] Average assets,[Releases] Average asset downloads,[Releases] Average asset size,[Releases] Prereleases to total,[Releases] Per day created (True),[Releases] Per day published (True),[Releases] Downloads per day (True)
[Workflow Runs] Count,0.104460,0.207992,0.068096,0.219526,-0.028301,0.009673,0.050683,-0.026125,-0.045959,0.112423,0.100056,0.068858
[Workflow Runs] Successful to total,0.153161,0.098340,-0.010983,0.095094,0.119774,0.036149,0.122598,0.090045,0.039170,0.026164,0.031343,-0.010849
[Workflow Runs] Failed to total,-0.024448,0.046591,-0.077056,0.067591,-0.023721,-0.145781,0.032997,0.012205,0.140765,-0.001142,-0.009442,-0.070514
[Workflow Runs] Skipped to total,0.212499,0.001563,-0.012073,0.021415,-0.151354,-0.026422,-0.011280,-0.007239,0.018898,0.160618,0.126430,-0.016071
[Workflow Runs] Average duration (ms),0.098397,0.060403,-0.066703,0.023740,-0.150682,-0.110822,-0.030580,0.048200,0.021728,0.015513,-0.002920,-0.079991
[Workflow Runs] Average success duration (ms),0.074637,0.043386,-0.042576,0.058158,-0.119577,-0.082256,-0.029948,0.088100,0.082176,-0.010834,-0.024875,-0.054324
[Workflow Runs] Average failure duration (ms),0.094278,0.079656,-0.067715,0.008633,-0.078919,-0.088452,-0.038788,0.012419,0.011455,0.110444,0.088055,-0.076359
[Workflow Runs] Average successes per day,0.048780,0.407150,0.124070,0.266120,0.044988,0.023976,0.025970,-0.035154,0.057435,0.069118,0.068617,0.126195
[Workflow Runs] Average successes per day (True),0.024691,0.122896,0.112478,0.247271,0.037689,0.031784,0.067389,-0.030343,-0.061264,0.110893,0.116232,0.131874
[Workflow Runs] Average fails per day,0.128791,0.491544,-0.027202,0.213314,0.038486,-0.021566,-0.100011,-0.047437,0.167028,0.017995,0.004442,-0.042815


,[Stars] Count,[Stars] Per day,[Stars] Per day (True),[Stars] Maximum per day
[Stars] Count,1.000000,-0.099225,0.651929,0.049444
[Stars] Per day,-0.099225,1.000000,-0.161357,0.449332
[Stars] Per day (True),0.651929,-0.161357,1.000000,-0.024386
[Stars] Maximum per day,0.049444,0.449332,-0.024386,1.000000


,[Stars] Count,[Stars] Per day,[Stars] Per day (True),[Stars] Maximum per day
[Workflow Runs] Count,0.040180,-0.071196,0.037588,-0.040650
[Workflow Runs] Successful to total,0.150502,0.083404,0.106786,-0.015408
[Workflow Runs] Failed to total,0.039834,-0.115701,0.006058,-0.051192
[Workflow Runs] Skipped to total,0.054927,-0.067751,0.049372,-0.048199
[Workflow Runs] Average duration (ms),-0.060684,-0.052901,-0.088558,0.075509
[Workflow Runs] Average success duration (ms),-0.061509,-0.077719,-0.082382,-0.014739
[Workflow Runs] Average failure duration (ms),-0.028306,-0.072441,-0.047239,-0.019673
[Workflow Runs] Average successes per day,-0.001451,-0.054198,-0.002346,0.007506
[Workflow Runs] Average successes per day (True),-0.032937,-0.111594,0.005321,-0.105376
[Workflow Runs] Average fails per day,0.054259,0.046126,0.016344,0.180944


,[Workflow Runs] Count,[Workflow Runs] Successful to total,[Workflow Runs] Failed to total,[Workflow Runs] Skipped to total,[Workflow Runs] Average duration (ms),[Workflow Runs] Average success duration (ms),[Workflow Runs] Average failure duration (ms),[Workflow Runs] Average successes per day,[Workflow Runs] Average successes per day (True),[Workflow Runs] Average fails per day,[Workflow Runs] Average fails per day (True)
[Workflow Runs] Count,1.000000,0.143129,-0.017447,0.520822,-0.031382,-0.022729,-0.032716,0.898855,0.862165,0.370919,0.612917
[Workflow Runs] Successful to total,0.143129,1.000000,0.151572,-0.056528,-0.439023,-0.351926,-0.486775,0.013314,0.052734,-0.364081,-0.240836
[Workflow Runs] Failed to total,-0.017447,0.151572,1.000000,0.030790,0.383788,0.251813,0.500349,-0.153841,-0.175106,0.262568,0.121401
[Workflow Runs] Skipped to total,0.520822,-0.056528,0.030790,1.000000,0.068190,0.072820,0.078471,0.230261,0.144993,0.266540,0.245804
[Workflow Runs] Average duration (ms),-0.031382,-0.439023,0.383788,0.068190,1.000000,0.880455,0.767439,0.000515,-0.050973,0.453679,0.064183
[Workflow Runs] Average success duration (ms),-0.022729,-0.351926,0.251813,0.072820,0.880455,1.000000,0.529052,0.025090,-0.036396,0.380697,0.018363
[Workflow Runs] Average failure duration (ms),-0.032716,-0.486775,0.500349,0.078471,0.767439,0.529052,1.000000,0.002308,-0.048903,0.460585,0.124449
[Workflow Runs] Average successes per day,0.898855,0.013314,-0.153841,0.230261,0.000515,0.025090,0.002308,1.000000,0.866252,0.515105,0.589609
[Workflow Runs] Average successes per day (True),0.862165,0.052734,-0.175106,0.144993,-0.050973,-0.036396,-0.048903,0.866252,1.000000,0.278434,0.665407
[Workflow Runs] Average fails per day,0.370919,-0.364081,0.262568,0.266540,0.453679,0.380697,0.460585,0.515105,0.278434,1.000000,0.567321
